In [19]:
import json
import numpy as np
from matplotlib import pyplot as plt
import os
from constants import LABEL_MAPPING
from ultralytics import YOLO
import math

In [8]:
detection_model = YOLO('../../data/detection_model/weights/last.pt')

In [37]:
def calculate_euclidean_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def compare_keypoints(ground_truth, predictions):
    if len(ground_truth) != len(predictions):
        raise ValueError("Input arrays must have the same length.")

    num_keypoints = len(ground_truth) - 2  # Excluding bounding box coordinates

    correct_keypoints = 0
    total_closeness = 0

    for i in range(2):
        gt_bbox = ground_truth[i]
        pred_bbox = predictions[i]
        distance = calculate_euclidean_distance(gt_bbox, pred_bbox)
        total_closeness += distance
        
    for i in range(num_keypoints):
        gt_keypoint = ground_truth[i + 2]
        pred_keypoint = predictions[i + 2]

        if (gt_keypoint == [0, 0] and pred_keypoint == [0, 0]) or (gt_keypoint != [0, 0] and pred_keypoint != [0, 0]):
            correct_keypoints += 1
            distance = calculate_euclidean_distance(gt_keypoint, pred_keypoint)
            total_closeness += distance

    accuracy = correct_keypoints / num_keypoints if num_keypoints > 0 else 1.0
    average_closeness = total_closeness / correct_keypoints if correct_keypoints > 0 else 0.0

    return accuracy, average_closeness

In [42]:
IMAGE_WIDTH = 1920
IMAGE_HEIGHT = 1080

# Opening JSON file
f = open('../../data/test/images/annotations.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)

total_accuracy = 0
total_closeness = 0
total_items = 0

for item in data['items']:
    sorted_annotations = sorted(item['annotations'], key=lambda x: x['label_id'])

    keypoints_actual = []
    keypoints_prediction =[]

    xtl = int(sorted_annotations[0]['bbox'][0])
    ytl = int(sorted_annotations[0]['bbox'][1])
    w = int(sorted_annotations[0]['bbox'][2])
    h = int(sorted_annotations[0]['bbox'][3])
    xbr = xtl + w 
    ybr = ytl + h

    keypoints_actual.append([xtl, ytl])
    keypoints_actual.append([xbr, ybr])

    for i in range(1, 17):
        px = int(sorted_annotations[i]['points'][0])
        py = int(sorted_annotations[i]['points'][1])
        visible = 0 if sorted_annotations[i]['attributes']['occluded'] == True else 2
        # Switch append to view a specific feature using LABEL_MAPPING
        if visible != 2:
            keypoints_actual.append([0,0])
        else:
            keypoints_actual.append([px, py])


    results = detection_model('../../data/test/images/%s.png' % sorted_annotations[0]['label_id'], verbose=False)

    for result in results:
            boxes = result.boxes 
            keypoints = result.keypoints
            numpy_boxes = boxes.numpy()
            numpy_keypoints = keypoints.numpy()
  
            for box in numpy_boxes[0]:
                xtl = int(box.xyxy[0][0])
                ytl = int(box.xyxy[0][1])
                xbr = int(box.xyxy[0][2])
                ybr = int(box.xyxy[0][3])

            keypoints_prediction.append([xtl, ytl])
            keypoints_prediction.append([xbr, ybr])

            for keypoints in numpy_keypoints:
                for index, p in enumerate(keypoints.xy[0], start=1):
                    x = int(p[0])
                    y = int(p[1])
                    keypoints_prediction.append([x,y])

    #print(keypoints_actual)
    #print(keypoints_prediction)

    accuracy, average_closeness = compare_keypoints(keypoints_actual, keypoints_prediction)
    
    total_accuracy += accuracy
    total_closeness += average_closeness
    total_items += 1
 
print(f"Average accuracy: {(total_accuracy/total_items) * 100:.2f}%")
print(f"Average closeness: {(total_closeness/total_items):.2f}")

# Closing file
f.close()

Average accuracy: 71.25%
Average closeness: 125.01
